### This notebook is produced for Coventry University's 7135CEM Coursework
Made by Sunggu Choi

The dataset is accessiable on https://www.kaggle.com/thoughtvector/customer-support-on-twitter

**Part 1**

Identitfy the topic is positive or negitive, also predict how the answer would be.

In [86]:
#!pip3 install python-Levenshtein
# !pip3 install tqdm
# !pip3 install textblob
# !pip3 install bokeh

     |████████████████████████████████| 10.6 MB 5.8 MB/s eta 0:00:01
  Created wheel for bokeh: filename=bokeh-2.3.1-py3-none-any.whl size=11299681 sha256=61e3fd821b18cf5b35439b2f6d6736fc7912b7e18eb160fd106258d670a200d8
  Stored in directory: /home/sunggu/.cache/pip/wheels/09/d1/a4/b53575bc2fa65fa5f52a29aafd6ed7b05f2ee5c11aa5a908ad
Successfully built bokeh


In [1]:
# Import requires libraries
import nltk
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm_notebook
import tqdm
import seaborn
from datetime import datetime
from matplotlib import pyplot as plt

#import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

# data visualisation and manipulation

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

Loading BokehJS ...

In [2]:
# Load dataset
df_dataset = pd.read_csv("./processed.csv")

In [3]:
# df_dataset["text"] = df_dataset["text"].map(lambda x: x.split(' '))

In [4]:
texts = [[word.lower() for word in text.split()] for text in df_dataset["text"]]

In [5]:
vect = TfidfVectorizer(stop_words=stop_words) # Whole corpus will be considered with max_feautre is set to None
word_matrix = vect.fit_transform(df_dataset['text']) # Topic Matrix 

In [6]:
idf=vect.idf_

In [7]:
words_dict=dict(zip(vect.get_feature_names(), idf)) # Generate dict 
keys_l=sorted(words_dict, key=(words_dict).get)

In [8]:
from sklearn.decomposition import TruncatedSVD

# n_components=100 == N_topic

n_components=100

lsa_model = TruncatedSVD(n_components=n_components, algorithm='randomized', n_iter=5, random_state=14)
lsa_topic_matrix =lsa_model.fit_transform(word_matrix)

In [9]:
# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

In [11]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_components):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words
# for i in range(len(top_n_words_lsa)):
#     print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
top_n_words_lsa = get_top_n_words(10, lsa_keys, word_matrix, vect)

In [ ]:
top_words = get_top_n_words(3, lsa_keys, word_matrix, vect)

In [ ]:
# Get 3 most frequest topics 


labels = ['Topic {}: \n'.format(i) +top_words[i] for i in lsa_categories]

fig, ax = plt.subplots(figsize=(20,10))
ax.bar(lsa_categories, lsa_counts);
ax.set_xticks(lsa_categories);
ax.set_xticklabels(labels);
ax.set_ylabel('Number of tweets');
ax.set_title('LSA topic counts');
plt.show()

In [ ]:
tsne_lsa_model = TSNE(n_components=5)
tsne_lsa_vectors = tsne_lsa_model.fit_transform(lsa_topic_matrix)

### VADER Sentiment Analysis
VADER uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labelled according to their semantic orientation as either positive or negative. 

In [3]:
# Install VADER
#!pip3 install vaderSentiment

     |████████████████████████████████| 125 kB 9.6 MB/s eta 0:00:01


In [7]:
df_dataset = pd.read_csv("./modified_ds.csv")

/home/sunggu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [16]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    
sentiment_analyzer_scores(df_dataset['text'][123])

@115752 Help is here. To assist you in recovering your account we recommend that you contact our Answer desk team: https://t.co/7zSM1QXPMD. {'neg': 0.0, 'neu': 0.785, 'pos': 0.215, 'compound': 0.6369}
